In [43]:
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform, color
from skimage.feature import hog
from skimage import exposure
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import os
from tqdm import tqdm
import cv2

import shutil

### downlaod the data from kaggke

In [2]:


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [3]:
!kaggle competitions download -c umor-classification

100% 192M/193M [00:11<00:00, 16.4MB/s]
100% 193M/193M [00:11<00:00, 17.6MB/s]


In [4]:
!unzip umor-classification.zip

Archive:  umor-classification.zip
  inflating: images/images/1006.png  
  inflating: images/images/1028.png  
  inflating: images/images/1046.png  
  inflating: images/images/1062.png  
  inflating: images/images/1066.png  
  inflating: images/images/1068.png  
  inflating: images/images/1070.png  
  inflating: images/images/1073.png  
  inflating: images/images/1091.png  
  inflating: images/images/1097.png  
  inflating: images/images/1105.png  
  inflating: images/images/1127.png  
  inflating: images/images/1138.png  
  inflating: images/images/1154.png  
  inflating: images/images/1156.png  
  inflating: images/images/1159.png  
  inflating: images/images/1183.png  
  inflating: images/images/1191.png  
  inflating: images/images/1202.png  
  inflating: images/images/1218.png  
  inflating: images/images/1230.png  
  inflating: images/images/1234.png  
  inflating: images/images/1235.png  
  inflating: images/images/1240.png  
  inflating: images/images/1253.png  
  inflating: ima

***

### read the image path

In [44]:
# # Load and preprocess training images
image_folder   = '/content/images/images/'

In [45]:
print(f"There are {len(os.listdir(image_folder ))} images in  dataset")

There are 775 images in  dataset


***

### read the csv files

In [46]:
train_df  = pd.read_csv('trainset.csv')
test_df   = pd.read_csv('testset.csv')

In [47]:
train_df['Class'].value_counts()

Benign       300
Malignant    148
Normal        91
Name: Class, dtype: int64

In [48]:
train_df

,Unnamed: 0,Id,Class
0,211,8602,Malignant
1,315,7464,Malignant
2,165,5832,Malignant
3,188,2843,Malignant
4,710,5272,Benign
...,...,...,...
534,763,8176,Benign
535,192,5434,Malignant
536,629,5366,Benign
537,559,9811,Benign


***
#### read the image folder

In [49]:
images = os.listdir(image_folder)

In [50]:
id_to_label_train = dict(zip(train_df['Id'], train_df['Class']))

***
##### read the train folder

In [51]:
# create a list for both images and labels
images_train = []
labels_train = []

In [52]:
for index, row in train_df.iterrows():
  im_id = row['Id']
  if im_id in id_to_label_train :
    label = id_to_label_train [im_id]
    image_path = os.path.join(image_folder, f"{im_id}.png")
    if os.path.exists(image_path):  # Check if the image file exists
        image = cv2.imread(image_path)  # Load the image using cv2

        if image is not None:  # Check if the image was loaded successfully
            images_train.append(image)
            labels_train.append(label)
            print(labels_train)

['Malignant']
['Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign', 'Benign', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignan

In [53]:
X_train = np.array(images_train)
X_train.shape

<ipython-input-53-8020449d68be>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(images_train)


(539,)

In [54]:
y_train = np.array(labels_train)
y_train.shape

(539,)

***
##### read the test folder

In [55]:
# Get the image IDs from the test CSV
test_image_ids = list(test_df['Id'])

In [56]:
image_test = []
for index, row in test_df.iterrows():
  image_id = row['Id']
  if image_id in test_image_ids:
    image_path = os.path.join(image_folder, f"{image_id}.png")
    # Check if the image file exists
    if os.path.exists(image_path):
      # Load the image using cv2
      image = cv2.imread(image_path)
      image_test.append(image)





In [57]:
image_test

[array([[[151, 151, 151],
         [ 11,  11,  11],
         [  0,   0,   0],
         ...,
         [169, 169, 169],
         [170, 170, 170],
         [169, 169, 169]],
 
        [[  0,   0,   0],
         [  7,   7,   7],
         [  1,   1,   1],
         ...,
         [152, 152, 152],
         [149, 149, 149],
         [146, 146, 146]],
 
        [[  1,   1,   1],
         [  0,   0,   0],
         [138, 138, 138],
         ...,
         [153, 153, 153],
         [153, 153, 153],
         [152, 152, 152]],
 
        ...,
 
        [[ 48,  48,  48],
         [ 49,  49,  49],
         [ 50,  50,  50],
         ...,
         [ 42,  42,  42],
         [ 42,  42,  42],
         [ 42,  42,  42]],
 
        [[ 45,  45,  45],
         [ 46,  46,  46],
         [ 46,  46,  46],
         ...,
         [ 42,  42,  42],
         [ 44,  44,  44],
         [ 44,  44,  44]],
 
        [[ 43,  43,  43],
         [ 43,  43,  43],
         [ 43,  43,  43],
         ...,
         [ 43,  43,  43],
  

In [58]:
X_test = np.array(image_test)
X_test.shape

<ipython-input-58-71dd8b3a7a86>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(image_test)


(229,)

***
know more information about the data


In [59]:
# summarize dataset shape
print('Train', X_train.shape, y_train.shape)
print('Test', (X_test.shape))
# summarize dataset shape
print('Train for image', X_train[0].shape, y_train[0].shape)
print('Test for image', (X_test[0].shape))

Train (539,) (539,)
Test (229,)
Train for image (470, 564, 3) ()
Test for image (467, 551, 3)


In [60]:
type(X_train)

numpy.ndarray

***

to here we will start prepraring data
***


### resize the image

for train

In [61]:
target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_train and resize
for image in X_train:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
X_train = np.array(resized_images)
# Now X_train_resized contains the resized images
print("Shape of X_train_resized:", X_train.shape)

Shape of X_train_resized: (539, 200, 200, 3)


for test

In [62]:
target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_test and resize
for image in X_test:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
X_test = np.array(resized_images)
# Now X_test contains the resized images
print("Shape of X_test:", X_test.shape)

Shape of X_test: (229, 200, 200, 3)


***
shape min max std mean for X_train and X_test


In [63]:
# Now compute the minimum and maximum values
minimum_pixel_value_train = np.min(X_train)
maximum_pixel_value_train = np.max(X_train)
print("Train - Minimum Pixel Value:", minimum_pixel_value_train)
print("Train - Maximum Pixel Value:", maximum_pixel_value_train)

minimum_pixel_value_test = np.min(X_test)
maximum_pixel_value_test = np.max(X_test)
print("Test - Minimum Pixel Value:", minimum_pixel_value_test)
print("Test - Maximum Pixel Value:", maximum_pixel_value_test)

Train - Minimum Pixel Value: 0
Train - Maximum Pixel Value: 255
Test - Minimum Pixel Value: 0
Test - Maximum Pixel Value: 255


### split the train into train and val set

In [64]:
# Split data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [65]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_train, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train:
Class Benign: 243 samples
Class Malignant: 118 samples
Class Normal: 70 samples


In [66]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_val, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train:
Class Benign: 57 samples
Class Malignant: 30 samples
Class Normal: 21 samples


In [67]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((431, 200, 200, 3), (108, 200, 200, 3), (431,), (108,))

In [71]:
!pip install imbalanced-learn

In [82]:
import numpy as np
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Assuming X_train, X_val, X_test, y_train, y_val, y_test are loaded with your data

# Reshape and normalize the image data
X_train_scaled = X_train / 255.0
X_val_scaled = X_val / 255.0
X_test_scaled = X_test / 255.0

# Convert images to grayscale
X_train_gray = np.array([rgb2gray(image) for image in X_train_scaled])
X_val_gray = np.array([rgb2gray(image) for image in X_val_scaled])
X_test_gray = np.array([rgb2gray(image) for image in X_test_scaled])

# Reshape the grayscale images for SMOTE
num_samples_train, height, width = X_train_gray.shape
X_train_gray_reshaped = X_train_gray.reshape(num_samples_train, height * width)

# Apply SMOTE for oversampling the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_gray_reshaped, y_train)

# Data augmentation and feature extraction using HOG for training data
hog_features_train = []
y_train_augmented = []  # To store augmented labels
for i in range(X_train_resampled.shape[0]):
    image = X_train_resampled[i].reshape(height, width)  # Reshape image
    hog_features = hog(image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    hog_features_train.append(hog_features)

    # Apply data augmentation
    augmented_image = np.fliplr(image)  # Horizontal flip
    augmented_hog_features = hog(augmented_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    hog_features_train.append(augmented_hog_features)

    # Add original and augmented labels
    y_train_augmented.extend([y_train_resampled[i]] * 2)  # Two samples per original
y_train_augmented = np.array(y_train_augmented)

# Combine original and augmented features
hog_features_train = np.array(hog_features_train)

# Feature extraction using HOG for validation data
hog_features_val = []
for image in X_val_gray:
    hog_features = hog(image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    hog_features_val.append(hog_features)
hog_features_val = np.array(hog_features_val)

# Feature extraction using HOG for test data
hog_features_test = []
for image in X_test_gray:
    hog_features = hog(image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    hog_features_test.append(hog_features)
hog_features_test = np.array(hog_features_test)

# Scale features using Min-Max scaling
scaler = MinMaxScaler()
scaled_features_train = scaler.fit_transform(hog_features_train)
scaled_features_val = scaler.transform(hog_features_val)
scaled_features_test = scaler.transform(hog_features_test)

In [83]:
scaled_features_train.shape

(1458, 8100)

In [84]:
scaled_features_val.shape

(108, 8100)

In [85]:
scaled_features_test.shape

(229, 8100)

In [86]:
y_train_augmented.shape

(1458,)

In [88]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_train_augmented, return_counts=True)
print("Class Frequencies in y_train_augmented:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train_augmented:
Class Benign: 486 samples
Class Malignant: 486 samples
Class Normal: 486 samples


In [121]:
from sklearn.decomposition import PCA
# Apply PCA for dimensionality reduction
pca = PCA(n_components=1000)  # Experiment with the number of components
pca.fit(scaled_features_train)
pca_features_train = pca.transform(scaled_features_train)
pca_features_val = pca.transform(scaled_features_val)
pca_features_test = pca.transform(scaled_features_test)


In [127]:
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define your CatBoost parameters
catboost_params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'MultiClass',
    # 'task_type': 'GPU',  # Use GPU for training
    # 'devices': '0',  # Specify GPU device index
    'verbose': 100
}

# Create the CatBoostClassifier
model = CatBoostClassifier(**catboost_params)

# Train the model on your filtered and HOG-processed training data
model.fit(pca_features_train, y_train_augmented, early_stopping_rounds=50)

# # Plot learning curves
# iterations = np.arange(0, model.tree_count_, model.verbose)
# plt.plot(iterations, model.evals_result_['learn']['MultiClass'], label='Train')
# plt.plot(iterations, model.evals_result_['validation']['MultiClass'], label='Validation')
# plt.xlabel('Iterations')
# plt.ylabel('Logloss')
# plt.title('CatBoost Learning Curves')
# plt.legend()
# plt.show()

# Predict on test data
y_pred = model.predict(pca_features_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

# Generate classification report
class_names = ['Class 0', 'Class 1', 'Class 2']
classification_rep = classification_report(y_val, y_pred, target_names=class_names)
print("Classification Report:")
print(classification_rep)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

0:	learn: 1.0700891	total: 1.94s	remaining: 32m 19s
100:	learn: 0.3827732	total: 1m 4s	remaining: 9m 35s
200:	learn: 0.1706692	total: 2m 4s	remaining: 8m 13s
300:	learn: 0.0961344	total: 3m 7s	remaining: 7m 15s
400:	learn: 0.0598657	total: 4m 6s	remaining: 6m 8s
500:	learn: 0.0410361	total: 5m 6s	remaining: 5m 4s
600:	learn: 0.0302377	total: 6m 7s	remaining: 4m 4s
700:	learn: 0.0232474	total: 7m 8s	remaining: 3m 2s
800:	learn: 0.0185318	total: 8m 9s	remaining: 2m 1s
900:	learn: 0.0152989	total: 9m 11s	remaining: 1m
999:	learn: 0.0128594	total: 10m 11s	remaining: 0us
Test Accuracy: 0.68
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.74      0.75      0.75        57
     Class 1       0.59      0.53      0.56        30
     Class 2       0.61      0.67      0.64        21

    accuracy                           0.68       108
   macro avg       0.65      0.65      0.65       108
weighted avg       0.67      0.68      0.67       108

Con

In [122]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

svm_classifier = SVC(kernel='rbf', C=10,gamma= 0.01)
svm_classifier.fit(pca_features_train, y_train_augmented)

# Make predictions
y_pred = svm_classifier.predict(pca_features_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(y_val, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 0.6666666666666666
Confusion Matrix:
 [[49  6  2]
 [15 15  0]
 [12  1  8]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.64      0.86      0.74        57
   Malignant       0.68      0.50      0.58        30
      Normal       0.80      0.38      0.52        21

    accuracy                           0.67       108
   macro avg       0.71      0.58      0.61       108
weighted avg       0.69      0.67      0.65       108



In [95]:
# Define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']}

# Create an SVM classifier
svm_classifier = SVC()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, scoring='accuracy', cv=5)

# Fit the grid search to the data
grid_search.fit(scaled_features_train, y_train_augmented)

# Get the best parameters and best estimator from the grid search
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters:", best_params)

Best Parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [96]:
# Fit the best estimator on the training data
best_estimator.fit(scaled_features_train, y_train_augmented)

# Make predictions on the validation set
y_pred = best_estimator.predict(scaled_features_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(y_val, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 0.6851851851851852
Confusion Matrix:
 [[52  3  2]
 [16 14  0]
 [12  1  8]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.65      0.91      0.76        57
   Malignant       0.78      0.47      0.58        30
      Normal       0.80      0.38      0.52        21

    accuracy                           0.69       108
   macro avg       0.74      0.59      0.62       108
weighted avg       0.71      0.69      0.66       108



In [115]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_val, return_counts=True)
print("Class Frequencies in y_val:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_val:
Class Benign: 57 samples
Class Malignant: 30 samples
Class Normal: 21 samples


In [129]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_pred, return_counts=True)
print("Class Frequencies in y_val:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_val:
Class Benign: 58 samples
Class Malignant: 27 samples
Class Normal: 23 samples


In [131]:
# Make predictions
predictions = model.predict(pca_features_test)

In [132]:
submit = pd.read_csv('/content/sample_submition4.csv')
submit

,Id,Class
0,8560,A
1,7920,A
2,8993,A
3,8126,A
4,9912,A
...,...,...
224,8343,A
225,1255,A
226,4573,A
227,3919,A


In [133]:
submit['Id'] = test_df['Id']
submit['Class'] =predictions # our model predictions on the test dataset
submit

,Id,Class
0,8560,Benign
1,7920,Benign
2,8993,Benign
3,8126,Normal
4,9912,Benign
...,...,...
224,8343,Malignant
225,1255,Benign
226,4573,Benign
227,3919,Benign


In [135]:
submit.to_csv('sample_submission_eman_new_way2_try_7.csv', index=False)